In [7]:
import numpy as np
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
from icecream import ic

## n-puzzle

In [8]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [9]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def goal_test(state: np.ndarray) -> bool:
    c = 1
    for i in state:
        for j in i:
            if j != c:
                return False
            c += 1
            if c == len(state)**2:
                c = 0
    return True

def stateInList(state: np.ndarray, frontier: list[np.ndarray]) -> int:
    for i, element in enumerate(frontier):
        c = 0
        for x in range(PUZZLE_DIM):
            for y in range(PUZZLE_DIM):
                if state[x][y] == element[x][y]:
                    c += 1
        if c == PUZZLE_DIM**2:
            return i
    return -1


In [10]:
RANDOMIZE_STEPS = 100_000
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    #scelgo randomicamente un azione da quelle possibili e la effettuo
    state = do_action(state, choice(available_actions(state)))
state

Randomizing:   0%|          | 0/100000 [00:00<?, ?it/s]

array([[1, 5, 8],
       [3, 6, 7],
       [0, 4, 2]])

## depth-first approach

Memory consumption is high, we can't always find a solution

In [13]:
frontier = list()
visited_state = list()
cost_list = list()
cost = 0
for step in tqdm(range(2000)):
    visited_state.append(state)
    if goal_test(state):
        break
    for a in available_actions(state):
        new_state = do_action(state, a)
        new_cost = cost + 1
        if stateInList(new_state, visited_state) == -1 and stateInList(new_state, frontier) == -1:
            frontier.append(new_state)
            cost_list.append(new_cost)
    state = frontier.pop()
    cost = cost_list.pop()
efficiency = cost / (step + 1)
ic(goal_test(state), state, efficiency, len(frontier), len(visited_state))
None

  0%|          | 0/2000 [00:00<?, ?it/s]

ic| goal_test(state): False
    state: array([[5, 8, 1],
                  [4, 3, 6],
                  [2, 7, 0]])
    efficiency: 0.9735
    len(frontier): 1559
    len(visited_state): 2000


## breadth-first approach

Memory consumption is high, we can't always find a solution

In [14]:
frontier = list()
visited_state = list()
cost_list = list()
cost = 0
for step in tqdm(range(2000)):
    visited_state.append(state)
    if goal_test(state):
        break
    for a in available_actions(state):
        new_state = do_action(state, a)
        new_cost = cost + 1
        if stateInList(new_state, visited_state) == -1 and stateInList(new_state, frontier) == -1:
            frontier.append(new_state)
            cost_list.append(new_cost)
    state = frontier.pop(0)
    cost = cost_list.pop(0)
efficiency = cost / (step + 1)
ic(goal_test(state), state, efficiency, len(frontier), len(visited_state))
None

  0%|          | 0/2000 [00:00<?, ?it/s]

ic| goal_test(state): False
    state: array([[2, 4, 5],
                  [0, 3, 8],
                  [7, 6, 1]])
    efficiency: 0.0065
    len(frontier): 1148
    len(visited_state): 2000


## A*